In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

## 依照指示取出模型特定層的資訊

In [3]:
# Define model
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16* 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
model = TheModelClass()


### 取出 self.pool層兩次的輸出，包含：
* x = self.pool(F.relu(self.conv1(x)))
* x = self.pool(F.relu(self.conv2(x)))

In [4]:
outputs= []
def layer1_hook(module, input_, output):
    outputs.append(output)
    
model.pool.register_forward_hook(layer1_hook)

In [7]:
input_ = torch.randn(1, 3, 32, 32)
output = model(input_)

In [8]:
outputs

[tensor([[[[0.5464, 0.8685, 0.0000,  ..., 0.4094, 0.0000, 0.6085],
           [1.7310, 0.3702, 1.0664,  ..., 0.4045, 0.1098, 0.1931],
           [0.3908, 1.0620, 0.1869,  ..., 0.6435, 0.5298, 0.9478],
           ...,
           [0.7473, 0.0591, 0.4502,  ..., 0.6226, 0.5786, 1.7559],
           [1.0018, 0.3074, 0.5510,  ..., 0.9289, 1.3282, 1.0312],
           [0.5863, 0.2384, 0.3842,  ..., 1.1192, 0.5303, 0.9133]],
 
          [[1.1793, 0.0000, 1.0065,  ..., 0.9253, 0.2357, 0.1119],
           [0.7462, 0.0000, 0.7970,  ..., 0.7601, 0.4422, 0.8037],
           [0.0091, 1.0531, 0.0000,  ..., 0.6688, 0.5693, 0.3053],
           ...,
           [0.7426, 0.6921, 0.4359,  ..., 1.0061, 0.1709, 1.5326],
           [0.9463, 0.2207, 0.8502,  ..., 0.9388, 0.7121, 1.7736],
           [0.7064, 0.9022, 1.2773,  ..., 1.2025, 0.6899, 0.8239]],
 
          [[0.8838, 0.3754, 0.4579,  ..., 0.5956, 0.3789, 0.1621],
           [1.0392, 0.6854, 0.8839,  ..., 0.2168, 0.1990, 0.6295],
           [0.0774, 0.30

In [9]:
print(outputs[0].shape)
print(outputs[1].shape)

torch.Size([1, 6, 14, 14])
torch.Size([1, 16, 5, 5])


## 加入自定義 initialization fuction

#### 對所有Conv2D層使用自定義initialization function
* weight : nn.init.kaiming_normal_
* bias : 全部輸入1

In [10]:
from torch.nn import init

def weights_init(m):
    #classname = m.__class__.__name__
    if isinstance(m, nn.Conv2d):
        #torch.nn.init.xavier_uniform_(m.weight)
        nn.init.kaiming_normal_(m.weight.data,
                                    a=0,
                                    mode='fan_out',
                                    nonlinearity='relu')
        if m.bias is not None:
            m.bias.data.fill_(1)
            
model.apply(weights_init)

TheModelClass(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

#### 查看 conv層的bias是否皆為1

In [11]:
for name, parameters in model.named_parameters():
    if ('conv' in name) and ('bias' in name):
        print(name, parameters)
        print('\n')

conv1.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)


conv2.bias Parameter containing:
tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       requires_grad=True)


